In [54]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter("ignore")

In [55]:
df = pd.read_csv("real_fake_sample.csv")
df.head()

,Text,label
0,WATCH: Bill Maher Hilariously Explains Why Tr...,Fake
1,China defends ally Pakistan after Trump critic...,Real
2,"Trudeau Does What Trump Can’t, Calls Out Whit...",Fake
3,WATCH: Trump Crudely HUMILIATED A Woman Onsta...,Fake
4,U.S. lawmakers seek looser energy development ...,Real


In [57]:
df.drop_duplicates(inplace = True)

**Data Cleaning**

In [32]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()

In [60]:
def title(item):
    rf = re.sub("[^A-Za-z0-9]", " ", item).lower().split()
    rf = [wnl.lemmatize(i) for i in rf if not i in set(stopwords.words("english"))]
    rf = " ".join(rf)
    rf = rf.split()[:20]
    rf = " ".join(rf).capitalize()
    return rf

In [64]:
def cleaning(item):
    rf = re.sub("[^A-Za-z0-9]", " ", item).lower().split()
    rf = [wnl.lemmatize(i) for i in rf if not i in set(stopwords.words("english"))]
    rf = " ".join(rf)
    return rf

In [61]:
df["Title"] = df["Text"].apply(title)

In [65]:
df["Clean_Text"] = df["Text"].apply(cleaning)

In [68]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
feature_vectors = tfidf.fit_transform(df["Clean_Text"])

In [70]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(feature_vectors)

In [201]:
r = df["Title"].str.split().apply(lambda x : x[:4])
df["Keywords"] = r.apply(lambda x: " ".join(x))

In [202]:
list_of_all_titles = df["Keywords"].tolist()

# Movie Name

In [203]:
name = input("Enter")

try:
    
    import difflib
    find_close_match = difflib.get_close_matches(name, list_of_all_titles)

    close_match = find_close_match[0]
    
    
    index_of_news = df[df["Keywords"] == close_match].index[0]
    
    similarity_score = enumerate(similarity[index_of_news])
    
    sorted_news = sorted(similarity_score, key = lambda x : x[1], reverse = True)
    
    print("News Suggested on these topics :")
    
    i = 0
    
    for m in sorted_news:
        index = m[0]
        title_from_index = df[df.index == index]["Keywords"].values[0]
        # title_from_index = df[df.index == index]["Title"].values[0]
        if (i <= 10):
            print(i, ".", title_from_index)
            i += 1
        if i == 10:
            break
            
except IndexError:
    extra_list = ['Fake Watch bill maher hilariously',
     'Real China defends ally pakistan',
     'Fake Trudeau trump call white',
     'Fake Watch trump crudely humiliated',
     'Real U lawmaker seek looser',
     'Fake Trump suggests banning moderator',
     'Fake Fox news host jesse',
     'Real Trump hold town hall',
     'Real Senate republican complain chaos',
     'Real Deputy attorney general seek']

    import difflib
    import random
    find_close_match = difflib.get_close_matches(random.choice(extra_list), list_of_all_titles)

    close_match = find_close_match[0]
    
    
    index_of_news = df[df["Keywords"] == close_match].index[0]
    
    similarity_score = enumerate(similarity[index_of_news])
    
    sorted_news = sorted(similarity_score, key = lambda x : x[1], reverse = True)
    
    print("News Suggested on these topics :")
    
    i = 0
    
    for m in sorted_news:
        index = m[0]
        title_from_index = df[df.index == index]["Keywords"].values[0]
        # title_from_index = df[df.index == index]["Title"].values[0]
        if (i <= 10):
            print(i, ".", title_from_index)
            i += 1
        if i == 10:
            break

Enter donald trump china russia


News Suggested on these topics :
0 . Donald trump whine swears
1 . Trump team allegedly offering
2 . Trump say senate passed
3 . New hampshire cop intent
4 . Trump try throw michelle
5 . Watch joe biden took
6 . Report priebus withdraws plan
7 . U senator introduce bill
8 . Something weird nearly half
9 . Snowflake trump kick campaign


In [205]:
import pickle
with open("news_vectorizer", "wb") as f:
    pickle.dump(tfidf, f)

with open("news_cosine_similarity", "wb") as f:
    pickle.dump(similarity, f)

In [206]:
df.to_csv("news_recommendation.csv", index = [0])